In [ ]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)

## Load Data 

In [ ]:
df = pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv', low_memory=False)
df.head()

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

df = reduce_memory_usage(df, verbose=True)

## DATA UNDERSTANDING

In [ ]:
def check_df(dataframe, head=5):
    print(" SHAPE ".center(70,'-'))
    print('Rows: {}'.format(dataframe.shape[0]))
    print('Columns: {}'.format(dataframe.shape[1]))
    print(" TYPES ".center(70,'-'))
    print(dataframe.dtypes)
    print(" MISSING VALUES ".center(70,'-'))
    print(dataframe.isnull().sum())
    print(" DUPLICATED VALUES ".center(70,'-'))
    print(dataframe.duplicated().sum())
    print(" DESCRIBE ".center(70,'-'))
    print(dataframe.describe().T)
    
check_df(df)

In [ ]:
df["overview"].head()

In [ ]:

tfidf = TfidfVectorizer(stop_words="english")


df['overview'] = df['overview'].fillna('')
df['overview'].isnull().sum()

In [ ]:

tfidf_matrix = tfidf.fit_transform(df['overview'])
tfidf_matrix.shape


In [ ]:
tfidf_matrix.toarray()


## Cosine Smilarity matrix

In [ ]:

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

cosine_sim.shape

In [ ]:

cosine_sim[1]

## Recommendation Based on Similarities 

In [ ]:


indices = pd.Series(df.index, index=df['title'])

indices.index.value_counts()

In [ ]:
indices = indices[~indices.index.duplicated(keep='last')]

indices["Cinderella"]

In [ ]:

movie_index = indices['Sherlock Holmes']

similarity_scores = pd.DataFrame(cosine_sim[movie_index],
                                 columns=["score"])

similarity_scores

In [ ]:

movie_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index

df['title'].iloc[movie_indices]


In [ ]:
def content_based_recommender(title, cosine_sim, dataframe):

    indices = pd.Series(dataframe.index, index=dataframe['title'])
    indices = indices[~indices.index.duplicated(keep='last')]
    
    movie_index = indices[title]
   
    similarity_scores = pd.DataFrame(cosine_sim[movie_index], columns=["score"])
    
    movie_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index
    return dataframe['title'].iloc[movie_indices]

content_based_recommender("Sherlock Holmes", cosine_sim, df)

In [ ]:
content_based_recommender("The Matrix", cosine_sim, df)

In [ ]:
content_based_recommender("The Godfather", cosine_sim, df)


In [ ]:
def calculate_cosine_sim(dataframe):
    tfidf = TfidfVectorizer(stop_words='english')
    dataframe['overview'] = dataframe['overview'].fillna('')
    tfidf_matrix = tfidf.fit_transform(dataframe['overview'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim
